In [2]:
import os
import pandas as pd
import re
import string
from nltk import tokenize


In [ ]:
#wiki scrambled
def find_keyword_sentences_scrambled(keywords, filename):               
    keyword_to_line = {}
    keyword_counter = {}
    count = 0
    #path to scrambled (by paragraph) Wikipedia data (change to your path)
    for subdir, dirs, files in os.walk('/juice/scr/katezhou/Summer2020/TwoWords/Section0/ScrambleWikipedia/wiki_scrambled/'):
        for file in files:
            count += 1
            text = open(os.path.join(subdir, file), 'r')
            for line in text:
                for word in line.split(" "):
                    if word in keywords:
                        value = keyword_to_line.get(word, [])
                        value.append(line)
                        keyword_to_line[word] = value

                        current_word_count = keyword_counter.get(word, 0)
                        keyword_counter[word] = current_word_count + 1
                        if (keyword_counter[word] >= 50):
                            keywords.remove(word)
                            if (len(keywords) % 100 == 0):
                                print(len(keywords))
                        #if word is found, don't continue with the same sentence to avoid the same
                        #sentence appearing multiple times throughout the corpus
                        #Limitation: keywords tend to exist at the start of the sentence
                        break
                                    
    lines = []
    keywords = []
    for keyword in keyword_to_line:
        for line in keyword_to_line[keyword]:
            lines.append(line)
            keywords.append(keyword)
    result = pd.DataFrame(lines, keywords)
    result.to_csv(filename)
    print("total number of files:", count)
    return


In [ ]:
#read in frequency data
word_freq = pd.read_csv("../utils/word_frequency/bert_word_frequencies.csv")
word_freq = word_freq[word_freq['freq'] >= 100].dropna()


In [ ]:
# grabbing words from the distribution
word_sample = word_freq[word_freq['freq'] < 1000][word_freq['freq'] >= 100].sample(30000)
word_sample = word_sample.append(word_freq[word_freq['freq'] < 10000][word_freq['freq'] >= 1000].sample(20000))
word_sample = word_sample.append(word_freq[word_freq['freq'] < 100000][word_freq['freq'] >= 10000].sample(10000))
word_sample = word_sample.append(word_freq[word_freq['freq'] >= 100000])

#save to file for reproducibility
# word_sample.to_csv("sentences/60K_random_words.csv")
keywords = set(list(temp['index'].values))


In [4]:
#read in sampled random words
keywords = pd.read_csv("sentences/60K_random_words.csv")
keywords = set(keywords['index'].values)

#find examples of such words in wikipedia
filename = '30K_random_words_sentences.csv'
find_keyword_sentences_scrambled(keywords, filename)

,Unnamed: 0,index,wiki_count,bookcorpus_count,freq,percent_change
0,2233312,pre-screening,130.0,6.0,136.0,1.046154
1,150426,allston,812.0,27.0,839.0,1.033251
2,597266,co-occurring,304.0,9.0,313.0,1.029605
3,2693036,stratfield,169.0,5.0,174.0,1.029586
4,2474633,schisms,549.0,31.0,580.0,1.056466
...,...,...,...,...,...,...
62957,3173421,youre,26.0,171504.0,171530.0,6597.307692
62958,3173852,yourself,13248.0,135217.0,148465.0,11.206597
62959,3174396,youth,264138.0,14056.0,278194.0,1.053215
62960,3187564,zealand,254689.0,1420.0,256109.0,1.005575


## WiC Sentences ## 

In [16]:
def get_og_data(filename):
    og_data = pd.read_csv("../Section_2/WiC/WiC_data/" + filename + "/" + filename + ".data.txt", sep = '\t', header=None)
    og_data.columns = ['og_keyword', 'og_pos', 'og_index', 'og_sentence1', 'og_sentence2']

    gold_labels = pd.read_csv("../Section_2/WiC/WiC_data/" + filename + "/" + filename + ".gold.txt", sep = '\t', header=None)
    gold_labels.columns=['human_label']
    gold_labels['human_label'] = gold_labels['human_label'].apply(lambda x: 0 if x == 'F' else 1)

    og_data = og_data.set_index(og_data.index).join(gold_labels.set_index(gold_labels.index))
    return og_data

In [17]:
keywords = set(get_og_data("train")['og_keyword'].values).union(set(get_og_data("dev")['og_keyword'].values))
len(keywords)

1635

In [ ]:
filename = 'wic_sentences_wikipedia_corpus.csv'
find_keyword_sentences_scrambled(keywords, filename)